In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Practice').getOrCreate()

22/03/09 16:29:50 WARN Utils: Your hostname, VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/09 16:29:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 16:29:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training = spark.read.csv('test3.csv', header=True,inferSchema=True)
training.show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
| Agus| 31|        10| 30000|
| Budi| 30|         8| 25000|
|Cecep| 29|         4| 20000|
| Dede| 24|         3| 20000|
|Erwin| 21|         1| 15000|
| Fani| 23|         2| 18000|
+-----+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age", "Experience"], outputCol="Independent Features")

In [7]:
temp_data = featureassembler.transform(training)
temp_data.show()

+-----+---+----------+------+--------------------+
| Name|Age|Experience|Salary|Independent Features|
+-----+---+----------+------+--------------------+
| Agus| 31|        10| 30000|         [31.0,10.0]|
| Budi| 30|         8| 25000|          [30.0,8.0]|
|Cecep| 29|         4| 20000|          [29.0,4.0]|
| Dede| 24|         3| 20000|          [24.0,3.0]|
|Erwin| 21|         1| 15000|          [21.0,1.0]|
| Fani| 23|         2| 18000|          [23.0,2.0]|
+-----+---+----------+------+--------------------+



In [8]:
finalized_data = temp_data.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [9]:
from pyspark.ml.regression import LinearRegression
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

22/03/09 16:30:06 WARN Instrumentation: [324f1db4] regParam is zero, which might cause numerical instability and overfitting.
22/03/09 16:30:07 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/09 16:30:07 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/03/09 16:30:07 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [10]:
regressor.coefficients

DenseVector([-258.8832, 1670.0508])

In [11]:
regressor.intercept

20543.147208121274

In [12]:
# prediction
pred_results = regressor.evaluate(test_data)

In [13]:
pred_results.predictions.show()

/home/mukhlishga/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|16776.649746192812|
|          [24.0,3.0]| 20000|19340.101522842593|
+--------------------+------+------------------+



In [14]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1218.2741116751095, 1795975.1604008244)

In [15]:
spark.stop()